In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from pandarallel import pandarallel
from tqdm.auto import tqdm
tqdm.pandas()
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
pandarallel.initialize(progress_bar=True, shm_size_mb=20000)

os.makedirs("classified", exist_ok=True)
analyzer = SentimentIntensityAnalyzer()
files = sorted(glob.glob("data/nz_tweets_*.csv"))
#print(files)
pd.set_option('display.max_colwidth', -1)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


New pandarallel memory created - Size: 20000 MB
Pandarallel will run on 40 workers


In [2]:
%%time
def classify(row):
    soup = BeautifulSoup(row.html, "lxml")
    s = []
    for child in soup.find("p").children:
        if child.name == None:
            s.append(child)
        elif child.name == "img":
            s.append(child["alt"])
        else:
            s.append(child.text)
    text_with_emoji = " ".join(s)
    row["text_with_emoji"] = text_with_emoji
    vs = analyzer.polarity_scores(text_with_emoji)
    for k,v in vs.items():
        row[k] = v
    return row

for f in tqdm(files):
    df = pd.read_csv(f, sep=";")
    df = df.parallel_apply(classify, axis=1)
    print("Complete - writing csv")
    new_filename = "classified/" + os.path.splitext(os.path.basename(f))[0] + ".csv"
    df.to_csv(new_filename, sep=";", index=False)

Complete - writing csv

CPU times: user 2min 56s, sys: 28.1 s, total: 3min 24s
Wall time: 6min 44s
